# Lesson-04

**Написать приложение, которое собирает основные новости с сайтов news.mail.ru, lenta.ru, yandex.news. Для парсинга использовать xpath. Структура данных должна содержать:**

**1. Название источника**

**2. Наименование новости**

**3. Ссылку на новость**

**4. Дата публикации**

In [349]:
from lxml import html
from requests import get
from pprint import pprint
import re
from datetime import datetime as dt
from pymongo import MongoClient

In [246]:
header = {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36"}

**lenta.ru**

In [247]:
def request_to_lenta():
    response = get('https://lenta.ru/',
                 headers = header
                 )
    
    root = html.fromstring(response.text)
    result = []
    items = root.xpath("//section[@class='row b-top7-for-main js-top-seven']//div[@class='item'] | //section[@class='row b-top7-for-main js-top-seven']//div[@class='first-item']")

    for item in items:
        info = {}
        name_source = 'https://lenta.ru/'
        name_news = item.xpath(".//a[not(@class)]/text() | .//a[@class='b-link-external']/text()")[0]
        name_news = name_news.replace('\xa0', ' ')
        link = item.xpath(".//a[not(@class)]/@href | .//a[@class='b-link-external']/@href")[0]
        date = item.xpath(".//a[not(@class)]//@datetime | .//a[@class='b-link-external']//@datetime")[0]
        date = re.findall(r'\d+\:\d+', date)[0]
        date = dt.strptime(date+ ' ' + str(dt.today().date()), '%H:%M %Y-%m-%d')

        info['source'] = name_source
        info['name'] = name_news
        info['link'] = link
        info['datetime'] = date
        result.append(info)
    return result

**yandex.news**

In [249]:
def request_to_yandex():
    yandex_link = 'https://yandex.ru'
    response = get(yandex_link + '/news',
                 headers = header
                 )
    
    root = html.fromstring(response.text)
    result = []
    items = root.xpath("//div[@class='page-content__fixed page-content__fixed_middle']/div/table//td")
    
    for item in items:
        info = {}
        name_source = item.xpath(".//div[@class='story__info']/div[@class='story__date']/text()")[0]
        name_news = item.xpath(".//h2/a/text()")[0]
        link = item.xpath(".//h2/a//@href")[0]
        date = re.findall(r'\d+\:\d+', name_source)[0]
        name_source = name_source.replace(' ' + date, '')
        date = dt.strptime(date+ ' ' + str(dt.today().date()), '%H:%M %Y-%m-%d')
        
        
        info['source'] = name_source
        info['name'] = name_news
        info['link'] = yandex_link + link
        info['datetime'] = date
        result.append(info)
    return result

**news.mail.ru**

In [346]:
def request_to_mail():
    mail_link = 'https://news.mail.ru/'
    response = get(mail_link,
                 headers = header
                 )
    
    root = html.fromstring(response.text)
    result = []
    items = root.xpath("//div[@class='js-module']/div//td/div[@class='daynews__item daynews__item_big'] | //div[@class='js-module']/div//td/div[@class='daynews__item']")
    
    for item in items:
        info = {}
        name_source = item.xpath(".//a/@href")[0]
        

        response_sub = get(mail_link + name_source,
                    headers = header
                    )
        if response_sub.ok:
            info['link'] = mail_link + name_source
            root_sub = html.fromstring(response_sub.text)
            result_sub = []
            date = root_sub.xpath("//div[@class='article js-article js-module']/div/span//@datetime")[0]
            date = re.findall(r'\d+\-\d+\-\w+\:\d+', date)[0]
            date = date.replace('T', ' ')
            date = dt.strptime(date, '%Y-%m-%d %H:%M')
            info['datetime'] = date
            name = root_sub.xpath("//div[@class='article js-article js-module']/div//h1/text()")[0]
            info['name'] = name
            source = root_sub.xpath("//span[@class='breadcrumbs__item']//span[@class='link__text']/text()")[0]
            info['source'] = source
            
        else:
            response_sub = get(name_source,
                            headers = header
                             ) 
            root_sub = html.fromstring(response_sub.text)
            info['link'] = name_source
            result_sub = []
            date = root_sub.xpath("//div[@class='article js-article']//span//@datetime")[0]
            date = re.findall(r'\d+\-\d+\-\w+\:\d+', date)[0]
            date = date.replace('T', ' ')
            date = dt.strptime(date, '%Y-%m-%d %H:%M')
            info['datetime'] = date
            name = root_sub.xpath("//h1/text()")[0]
            info['name'] = name
            source = root_sub.xpath("//span[@class='breadcrumbs__item']//span[@class='link__text']/text()")[0]
            info['source'] = source
        
        result.append(info)
    return result

**Сложить все новости в БД**

In [351]:
lenta = request_to_lenta()
pprint(lenta)

[{'datetime': datetime.datetime(2020, 4, 19, 16, 40),
  'link': '/news/2020/04/19/armata/',
  'name': 'Россия испытала танк «Армата» в Сирии',
  'source': 'https://lenta.ru/'},
 {'datetime': datetime.datetime(2020, 4, 19, 17, 49),
  'link': '/news/2020/04/19/baxily/',
  'name': 'Российская чиновница отчитала медиков за нежелание шить себе бахилы',
  'source': 'https://lenta.ru/'},
 {'datetime': datetime.datetime(2020, 4, 19, 17, 29),
  'link': '/news/2020/04/19/prognoz/',
  'name': 'Названы сроки возврата к нормальной жизни после эпидемии '
          'коронавируса в России',
  'source': 'https://lenta.ru/'},
 {'datetime': datetime.datetime(2020, 4, 19, 17, 26),
  'link': '/news/2020/04/19/dog/',
  'name': 'Мужчина нашел на шоссе собаку с трогательной запиской и расплакался',
  'source': 'https://lenta.ru/'},
 {'datetime': datetime.datetime(2020, 4, 19, 17, 11),
  'link': '/news/2020/04/19/aurus/',
  'name': 'Стали известны сроки первой поставки машин из «Кортежа» Путина',
  'source': '

In [352]:
yandex = request_to_yandex()
pprint(yandex)

[{'datetime': datetime.datetime(2020, 4, 19, 17, 34),
  'link': 'https://yandex.ru/news/story/V_Peterburge_razrabotali_prilozhenie_dlya_progulok_bez_shtrafov--cc03b67fd8d82de9b190748d8501cfa8?lr=2&lang=ru&stid=S7pEifoDLX1lIBrg8S_b&persistent_id=94921675&rubric=Saint_Petersburg&from=index',
  'name': 'В Петербурге разработали приложение для прогулок без штрафов',
  'source': 'КП - Санкт-Петербург'},
 {'datetime': datetime.datetime(2020, 4, 19, 16, 15),
  'link': 'https://yandex.ru/news/story/62_blokposta_poyavyatsya_v_Peterburge_v_sluchae_zakrytiya_goroda--b46a42a8407ffd9071ce2d98ec0871fb?lr=2&lang=ru&stid=8iC3HEjs_dKg0w1dtO2B&persistent_id=94878022&rubric=Saint_Petersburg&from=index',
  'name': '62 блокпоста появятся в Петербурге в случае закрытия города',
  'source': 'Карповка'},
 {'datetime': datetime.datetime(2020, 4, 19, 17, 30),
  'link': 'https://yandex.ru/news/story/Nezakonnyj_krestnyj_khod_v_centre_Peterburga_prervala_policiya--6a8a64cb2f6d2b7fb0b42d350a15f1c1?lr=2&lang=ru&stid

In [353]:
mail = request_to_mail()
pprint(mail)

[{'datetime': datetime.datetime(2020, 4, 19, 12, 14),
  'link': 'https://news.mail.ru//society/41450650/',
  'name': 'В России впервые выявлено более 6000 больных коронавирусом за сутки',
  'source': 'Ведомости'},
 {'datetime': datetime.datetime(2020, 4, 19, 17, 3),
  'link': 'https://news.mail.ru//society/41453467/',
  'name': 'Ученые назвали тревожный симптом коронавируса',
  'source': 'РИА Новости'},
 {'datetime': datetime.datetime(2020, 4, 19, 17, 21),
  'link': 'https://news.mail.ru//incident/41453715/',
  'name': 'Вору в законе подобрали «профессиональную» статью',
  'source': 'Коммерсантъ'},
 {'datetime': datetime.datetime(2020, 4, 19, 14, 38),
  'link': 'https://sportmail.ru/news/figure-skating/41450332/',
  'name': 'Уехавший в США русский фигурист Красножон рассказал о масштабах '
          'паники американцев во время пандемии коронавируса',
  'source': 'Sport24'},
 {'datetime': datetime.datetime(2020, 4, 19, 16, 43),
  'link': 'https://news.mail.ru//politics/41453332/',
  'n

In [350]:
client = MongoClient('localhost', 27017)
db = client['news']

In [354]:
def insert_news(site):
    for i in site:
        db.news.update_one({'link': i['link']}, 
                                     {'$set': i},
                                      upsert=True
                                      )

In [365]:
insert_news(lenta)
insert_news(yandex)
insert_news(mail)

In [366]:
objects = db.news.find( {} )
for obj in objects:
    pprint(obj)

{'_id': ObjectId('5e9c67aa794321e75f07e40f'),
 'datetime': datetime.datetime(2020, 4, 19, 16, 40),
 'link': '/news/2020/04/19/armata/',
 'name': 'Россия испытала танк «Армата» в Сирии',
 'source': 'https://lenta.ru/'}
{'_id': ObjectId('5e9c67aa794321e75f07e411'),
 'datetime': datetime.datetime(2020, 4, 19, 17, 49),
 'link': '/news/2020/04/19/baxily/',
 'name': 'Российская чиновница отчитала медиков за нежелание шить себе бахилы',
 'source': 'https://lenta.ru/'}
{'_id': ObjectId('5e9c67aa794321e75f07e413'),
 'datetime': datetime.datetime(2020, 4, 19, 17, 29),
 'link': '/news/2020/04/19/prognoz/',
 'name': 'Названы сроки возврата к нормальной жизни после эпидемии '
         'коронавируса в России',
 'source': 'https://lenta.ru/'}
{'_id': ObjectId('5e9c67aa794321e75f07e415'),
 'datetime': datetime.datetime(2020, 4, 19, 17, 26),
 'link': '/news/2020/04/19/dog/',
 'name': 'Мужчина нашел на шоссе собаку с трогательной запиской и расплакался',
 'source': 'https://lenta.ru/'}
{'_id': ObjectId(